In [1]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
o_router_key=userdata.get('open_router_test')

In [3]:
from openai import OpenAI
from os import getenv

# gets API Key from environment variable OPENAI_API_KEY
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=o_router_key
)

In [35]:
system_prompt = '''
You are a well equiped cs2/csgo item describer, your goal is to define cs2 skins and weapons in the screen in as much detail as possible, main focus is the color, theme, lore, story and all sorts of visual aspects.
'''

def analyze_image(img_url):
    response = client.chat.completions.create(
    model="gpt-4o-mini",
    # model="fireworks/firellava-13b",
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": img_url,
                    }
                },
            ],
        },
        {
            "role": "user",
            "content": "Define the given counter strike item standing infront of the screen or being held by a game character, colors, theme and all sorts of visual aspects. You don't need to count them one by one but as a general review. Do not lean towards your previous knowledge about item types, just describe it like you've no prior knowledge about skins, you can still use your history and mythological knowledge though. Do not focus on the background or the holder, just describe the item in question."
        }
    ],
        max_tokens=125,
        top_p=0.1
    )

    return response.choices[0].message.content

In [28]:
analyze_image("https://images.waxpeer.com/screenshot/items/37022837531_front_auto_1713616294403.webp")

"The item presented is a sniper rifle, characterized by a sleek and elongated design that emphasizes precision and power. The body of the rifle is predominantly a deep, rich blue, reminiscent of the ocean depths, which creates a striking contrast against the metallic silver components, such as the barrel and scope.\n\nIntricate designs adorn the rifle's surface, featuring swirling patterns that evoke the movement of water or tentacles, suggesting a connection to the sea. These patterns are highlighted with shades of green and purple, adding depth and a sense of mystique. The artwork includes a prominent face, possibly inspired by mythological figures, with flowing hair that"

In [29]:
# mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
# copy dir from drive to here

import shutil

shutil.copy("/content/drive/MyDrive/cs_images/images.zip", "/content/images.zip")

'/content/images.zip'

In [ ]:
!unzip images.zip

In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import time
import os
import argparse
from PIL import Image
from io import BytesIO

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
DELAY = 0.7
SAVE_DIR = "csgoskins/images"
ANALYSIS_FILE = "csgoskins/image_analysis.json"

def read_filtered_items(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

def fetch_image_urls(item_name, base_url):
    def fetch_url(url):
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        return BeautifulSoup(response.content, "html.parser")

    def is_valid_page(soup):
        return not soup.select_one("h1:-soup-contains('Page Not Found')")

    try:
        soup = fetch_url(base_url)
        if not is_valid_page(soup):
            return []  # Return empty list if page is not valid

        # Find the first image in the gallery
        image_element = soup.select_one("img.gallery-image")
        return image_element["data-download-url"] if image_element and "data-download-url" in image_element.attrs else None
    except requests.RequestException as e:
        print(f"Request error for {item_name}: {e}")
    except Exception as e:
        print(f"Error processing {item_name}: {e}")
    return None

def download_image(image_url, save_path, quality=30, max_size=(800, 800)):
    try:
        response = requests.get(image_url, headers=HEADERS)
        response.raise_for_status()

        # Open the image and process it
        img = Image.open(BytesIO(response.content))
        img.thumbnail(max_size, Image.ANTIALIAS)
        img.save(save_path, format="PNG", quality=quality)

        print(f"Downloaded and resized image to {save_path}")
    except requests.RequestException as e:
        print(f"Request error for image {image_url}: {e}")
    except Exception as e:
        print(f"Error downloading image {image_url}: {e}")

def save_analysis_results(results, file_path):
    with open(file_path, "w") as file:
        json.dump(results, file, indent=4)

def main(scrape_stickers):
    items = read_filtered_items("csgoskins/filtered_items.json")

    if scrape_stickers:
        items = read_filtered_items("csgoskins/filtered_stickers.json")

    print("length of items: ", len(items))

    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)

    analysis_results = {}

    for item in items:
        # Ensure that only items with "sticker" in the name or link are processed when scrape_stickers is True
        if scrape_stickers and (
            "sticker" not in item["name"].lower()
            and "sticker" not in item["link"].lower()
        ):
            continue
        try:
            print(f"Fetching images for: {item['name']}")
            image_url = fetch_image_urls(item["name"], item["link"])
            if image_url:
                item_dir = os.path.join(SAVE_DIR, item["name"].replace(" ", "_"))
                if not os.path.exists(item_dir):
                    os.makedirs(item_dir)
                save_path = os.path.join(item_dir, "1.png")
                download_image(image_url, save_path)
                # Analyze the image and save the result
                analysis_result = analyze_image(image_url)
                analysis_results[item["name"]] = {"1.png": analysis_result}
            else:
                print(f"No images found for {item['name']}")
            print()
            time.sleep(DELAY)
        except Exception as e:
            print(f"Error fetching images for {item['name']}: {e}")

    save_analysis_results(analysis_results, ANALYSIS_FILE)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Scrape CS:GO item images.")
    parser.add_argument(
        "--scrape-stickers",
        action="store_true",
        help="Only scrape stickers if this flag is set.",
    )
    args, unknown = parser.parse_known_args()
    main(args.scrape_stickers)


length of items:  1689
Fetching images for: M9 Bayonet | Doppler


<ipython-input-38-668a15c7abb8>:51: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img.thumbnail(max_size, Image.ANTIALIAS)


Downloaded and resized image to csgoskins/images/M9_Bayonet_|_Doppler/1.png

Fetching images for: Desert Eagle | Printstream
Downloaded and resized image to csgoskins/images/Desert_Eagle_|_Printstream/1.png

Fetching images for: AK-47 | Inheritance
Downloaded and resized image to csgoskins/images/AK-47_|_Inheritance/1.png

Fetching images for: Karambit | Doppler
Downloaded and resized image to csgoskins/images/Karambit_|_Doppler/1.png

Fetching images for: Driver Gloves | Imperial Plaid
Downloaded and resized image to csgoskins/images/Driver_Gloves_|_Imperial_Plaid/1.png

Fetching images for: Butterfly Knife | Autotronic
Downloaded and resized image to csgoskins/images/Butterfly_Knife_|_Autotronic/1.png

Fetching images for: AK-47 | Asiimov
Downloaded and resized image to csgoskins/images/AK-47_|_Asiimov/1.png
